In [2]:
%load_ext sql
import sqlite3

In [3]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

# 8. Demonstration Questions

## A. Do we need to increase the number of Intro to Accounting Courses offered due to the increase in student enrollment?

In [20]:
%%sql
SELECT CATALOG_ID, TERM, AVG(REMAINING), AVG(ACTUAL)
FROM COURSE_FACTS
WHERE CATALOG_ID = 'AC 0011'and TERM LIKE '%FALL%' and CAPACITY <> 0
GROUP BY TERM
ORDER BY TERM 

 * sqlite:///CourseDataWarehouse.db
Done.


CATALOG_ID,TERM,AVG(REMAINING),AVG(ACTUAL)
AC 0011,Fall2014,-1.2222222222222223,31.22222222222222
AC 0011,Fall2015,0.4444444444444444,28.555555555555557
AC 0011,Fall2016,1.7,26.4
AC 0011,Fall2017,1.2222222222222223,28.77777777777778
AC 0011,Fall2018,0.4,30.8


## B. Should we offer more classes on Monday's during a particular time code?

In [11]:
%%sql
SELECT DISTINCT TIMECODE, DAY, START, END, sum(ACTUAL), sum(CAPACITY), sum(REMAINING), count(CATALOG_ID) 
FROM COURSE_FACTS
    JOIN TIME_SLICES USING (TIMECODE)
WHERE DAY = 'M' and CAPACITY <> 0
GROUP BY TIMECODE
ORDER BY sum(REMAINING) ASC
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


TIMECODE,DAY,START,END,sum(ACTUAL),sum(CAPACITY),sum(REMAINING),count(CATALOG_ID)
162,M,14:00:00,17:00:00,130,114,-16,6
299,M,10:50:00,13:50:00,52,40,-12,2
756,M,09:00:00,12:15:00,48,36,-12,2
708,M,13:50:00,16:50:00,41,30,-11,2
547,M,18:30:00,22:15:00,30,20,-10,1


In [10]:
%%sql
SELECT DISTINCT TIMECODE, DAY, START, END, sum(ACTUAL), count(CATALOG_ID)
FROM COURSE_FACTS
    JOIN TIME_SLICES USING (TIMECODE)
WHERE DAY = 'M' and CAPACITY <> 0
GROUP BY TIMECODE
ORDER BY sum(ACTUAL) DESC
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


TIMECODE,DAY,START,END,sum(ACTUAL),count(CATALOG_ID)
8,M,11:00:00,12:15:00,13055,572
10,M,12:30:00,13:45:00,11735,512
28,M,14:00:00,15:15:00,11596,535
14,M,09:30:00,10:45:00,11553,515
12,M,15:30:00,16:45:00,9077,417


## C. How are rooms utilized in the Dolan School of Business?

In [16]:
%%sql
SELECT ROOM_ID, count(CATALOG_ID)
FROM COURSE_FACTS
    JOIN LOCATIONS USING (LOCATION_ID)
WHERE ROOM_ID LIKE '%DSB%'
GROUP BY ROOM_ID
ORDER BY count(CATALOG_ID)
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


ROOM_ID,count(CATALOG_ID)
DSB 110,4
DSB 2109A,145
DSB 110B,246
DSB 110A,247
DSB 114,264


In [14]:
%%sql
SELECT ROOM_ID, count(CATALOG_ID)
FROM COURSE_FACTS
    JOIN LOCATIONS USING (LOCATION_ID)
WHERE ROOM_ID LIKE '%DSB%'
GROUP BY ROOM_ID
ORDER BY count(CATALOG_ID) DESC
LIMIT 5;

 * sqlite:///CourseDataWarehouse.db
Done.


ROOM_ID,count(CATALOG_ID)
DSB 109,398
DSB 112,390
DSB 106,359
DSB 115,352
DSB 108,350


## D. Which departments are in need of more professors to help advise students?

In [104]:
%%sql
SELECT PROGRAM_NAME, count(DISTINCT INSTRUCTOR_ID), sum(ACTUAL), sum(ACTUAL)/count(DISTINCT INSTRUCTOR_ID) as RATIO
FROM COURSE_FACTS
    JOIN INSTRUCTORS USING (INSTRUCTOR_ID)
    JOIN PROGRAMS USING (PROGRAM_ID)
WHERE TERM LIKE '%Spring2019%' 
GROUP BY PROGRAM_NAME, TERM
ORDER BY RATIO DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


PROGRAM_NAME,count(DISTINCT INSTRUCTOR_ID),sum(ACTUAL),RATIO
Physics,6,991,165
Operations Management,4,648,162
Economics,12,1811,150
Marketing,11,1639,149
Sociology,7,999,142
Mathematics,24,3312,138
Accounting,19,2479,130
Religious Studies,16,2015,125
Management,13,1593,122
Spanish,17,2087,122


## E. What are the least popular classes? Should we consider removing them from the catalog? 

In [84]:
%%sql
SELECT CATALOG_ID, TITLE, PROGRAM_NAME,TERM, sum(ACTUAL)
FROM COURSE_FACTS
    JOIN PROGRAMS USING (PROGRAM_ID)
WHERE TERM LIKE '%2018%'
GROUP BY CATALOG_ID
ORDER BY sum(ACTUAL) ASC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


CATALOG_ID,TITLE,PROGRAM_NAME,TERM,sum(ACTUAL)
BI 0396,Biology Research VI,Biology,Fall2018,0
ECE 0448,Embedded Microcontrollers,Electrical and Computer Engineering,Fall2018,0
ECE 0448L,Embedded Microcontrollers Lab,Electrical and Computer Engineering,Fall2018,0
AH 0330A,Senior Capstone Seminar,Art History,Spring2018,1
CL 0109,Greek Tragedy in English Translation,Classical Studies,Spring2018,1
ED 0463,World Language Methods,Education,Fall2018,1
PS 0392,Theoretical/Experimental Capstone,Physics,Spring2018,1
BI 0394,Biology Research IV,Biology,Fall2018,2
CO 0497O,Conflict Communication,Communication,Spring2018,2
ECE 0592,Capstone Professional Project II,Electrical and Computer Engineering,Fall2018,2
